In [24]:
# based on:
# https://open-meteo.com/en/docs/historical-weather-api

In [25]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


In [26]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [27]:

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2004-10-03",
	"end_date": "2004-10-17",
	"daily": ["weather_code", "rain_sum", "sunshine_duration"]
}
responses = openmeteo.weather_api(url, params=params)


In [28]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [29]:
daily = response.Daily()
weathers_code = daily.Variables(0).ValuesAsNumpy()
rain_sum = daily.Variables(1).ValuesAsNumpy()
sunshine_duration = daily.Variables(2).ValuesAsNumpy()


In [30]:

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weathers_code"] = weathers_code
daily_data["rain_sum"] = rain_sum
daily_data["sunshine_duration"] = sunshine_duration


In [31]:
daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)


                        date  weathers_code  rain_sum  sunshine_duration
0  2004-10-03 00:00:00+00:00            2.0       0.0       36037.921875
1  2004-10-04 00:00:00+00:00            3.0       0.0       28481.058594
2  2004-10-05 00:00:00+00:00            1.0       0.0       36000.000000
3  2004-10-06 00:00:00+00:00           53.0       1.0       18992.345703
4  2004-10-07 00:00:00+00:00            3.0       0.0       35736.089844
5  2004-10-08 00:00:00+00:00            3.0       0.0       18751.058594
6  2004-10-09 00:00:00+00:00            3.0       0.0       29540.986328
7  2004-10-10 00:00:00+00:00            3.0       0.0       28730.789062
8  2004-10-11 00:00:00+00:00            0.0       0.0       35163.562500
9  2004-10-12 00:00:00+00:00            0.0       0.0       34953.515625
10 2004-10-13 00:00:00+00:00            3.0       0.0       29209.494141
11 2004-10-14 00:00:00+00:00            1.0       0.0       34442.671875
12 2004-10-15 00:00:00+00:00           53.0       2

In [32]:
daily_dataframe[daily_dataframe["weathers_code"] == 51.0]

,date,weathers_code,rain_sum,sunshine_duration
14,2004-10-17 00:00:00+00:00,51.0,1.3,29387.103516


In [33]:
daily_dataframe[daily_dataframe["rain_sum"] > 0.0]

,date,weathers_code,rain_sum,sunshine_duration
3,2004-10-06 00:00:00+00:00,53.0,1.0,18992.345703
12,2004-10-15 00:00:00+00:00,53.0,2.2,3548.996338
13,2004-10-16 00:00:00+00:00,53.0,3.5,1226.747070
14,2004-10-17 00:00:00+00:00,51.0,1.3,29387.103516


In [34]:
daily_dataframe["rain_sum"].value_counts()

rain_sum
0.0    11
1.0     1
2.2     1
3.5     1
1.3     1
Name: count, dtype: int64